# Тетрадка

тетрадка для подведения итогов расчета

In [ ]:
import os
import sys
sys.path.append('../../../')
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime
from multiprocessing import Pool

In [ ]:
import plot_workflow.plotly_option as pltl_opt
import plot_workflow.plotly_workflow as pltl_wf

from preproc_p import workflow_cs_data
from preproc_p import workflow_chess_data
from preproc_p import preproc_tool
from preproc_p import workflow_calc_data
from preproc_p import workflow_tr_data
from preproc_p import filtration
from proc_p import processor as proc

from ml import calibr_restore as calibr_restore
from postproc_p import result_and_metrics as result_and_metrics

In [ ]:
current_path = os.getcwd()
time_mark = '' #datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
path_to_data = current_path + "\\data\\"
dirnames_list = []
for (dirpath, dirnames, filenames) in os.walk(path_to_data):
    dirnames_list.extend(dirnames)
    break
print(dirnames_list)

In [ ]:
calculated_wells = ['1354', '1479', '1509', '1540', '1567', '1602', '1628', 
                    '252', '326', '353', '507', '540', '569', '570',
                    '601', '627', '658', '689', '693']

In [ ]:
metrics = []
overall_data_list = []
for well_name in calculated_wells:
    
    path_to_work_dir = current_path + "\\data\\" + well_name +  "\\"
    path_to_adapt_dir = 'adaptation_' + '\\'
    path_to_restore_dir = 'restore_' + '\\'
    
    adapt_data_with_input = pd.read_csv(path_to_work_dir + path_to_adapt_dir + well_name + '_calc_and_input' + '.csv' , parse_dates = True, index_col = 'Время')
    adapt_data_with_input = preproc_tool.mark_df_columns(adapt_data_with_input, 'ADAPT')
    restore_data_with_input = pd.read_csv(path_to_work_dir + path_to_restore_dir + well_name + '_calc_and_input' + '.csv' , parse_dates = True, index_col = 'Время')
    restore_data_with_input = preproc_tool.mark_df_columns(restore_data_with_input, 'RESTORE')
    overall_data = adapt_data_with_input.join(restore_data_with_input, how = 'outer')
    
    q_liq = overall_data[['Объемный дебит жидкости (СУ) (ADAPT)', 'Активная мощность (СУ) (ADAPT)']]
    result = preproc_tool.make_gaps_and_interpolate(q_liq)
    result = preproc_tool.mark_df_columns(result, 'INTERP')
    overall_data = overall_data.join(result)
    
    overall_data = result_and_metrics.final_edit_overall_data(overall_data)
    
    overall_data_list.append(overall_data)
    
    calibr_calc_metrics, interp_calc_metrics = result_and_metrics.calc_calibr_interp_metrics(overall_data)
    
    tr_name = "Техрежим, , февраль 2019.xls"
    
    tr_file_full_path = os.getcwd() + '\\data\\tr\\' + tr_name
    tr_data = workflow_tr_data.read_tr_and_get_data(tr_file_full_path, well_name)
    tr_data_df = pd.DataFrame({'Параметры скважины с ТР': list(tr_data.__dict__.values())})
    tr_data_df.index = list(tr_data.__dict__.keys())
    
    
    overall_metrics = result_and_metrics.calc_calibr_interp_metrics(overall_data, return_df = True)

    overall_metrics['ЭЦН'] = [tr_data.esp_name_str, tr_data.esp_name_str]
    overall_metrics['ПЭД'] = [tr_data.motor_name_str, tr_data.motor_name_str]
    overall_metrics['Кол-во точек (ADAPT)'] = [len(overall_data['К. калибровки по напору - множитель (Модель) (ADAPT)']), 
                                               len(overall_data['К. калибровки по напору - множитель (Модель) (ADAPT)'])]

    overall_metrics.index = [well_name + ' (CALIBR)' , well_name + ' (INTERP)']
    
    metrics.append(overall_metrics)

In [ ]:
result = metrics[0]
for i in metrics[1:]:
    result = result.append(i)
    

In [ ]:
result.to_excel('result.xlsx')

In [ ]:
result = result[result.index.str.contains('CALIBR')]

In [ ]:
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, plot, iplot
from plotly.subplots import make_subplots


In [ ]:
fig = make_subplots(
    rows=len(result.columns), cols=1)
for i in range(len(result.columns)):
    column_name = result.columns[i]
    x = result.index
    y = result[column_name]
    trace = go.Bar(x=x, y=y, name=column_name)
    i = i+1
    fig.add_trace(trace,
                  row=i, col=1)



fig.update_layout(height=len(result.columns) * 500,showlegend=True)
plot(fig, filename ='result' +  '.html')


Построение различных параметров для всех скважин на одном графике

In [ ]:
wells_result = overall_data_list[0][['К. калибровки по напору - множитель (Модель) (ADAPT)', 
                                    'К. калибровки по мощности - множитель (Модель) (ADAPT)',
                                    'Рлин ТМ (Ш) (ADAPT)', 'Линейное давление (СУ) (ADAPT)', 'Dшт (Ш) (ADAPT)']]
wells_result.columns = ['К. напор ' + calculated_wells[0] , 
                        'К. мощность ' + calculated_wells[0], 
                        'Рлин ТМ (Ш) (ADAPT) ' + calculated_wells[0], 
                        'Линейное давление (СУ) (ADAPT) ' + calculated_wells[0], 
                        'Dшт (Ш) (ADAPT) ' +  calculated_wells[0]]

In [ ]:
for i, j in zip(calculated_wells[1:], overall_data_list[1:]):
    new_result = j[['К. калибровки по напору - множитель (Модель) (ADAPT)', 
                        'К. калибровки по мощности - множитель (Модель) (ADAPT)',
                    'Рлин ТМ (Ш) (ADAPT)', 'Линейное давление (СУ) (ADAPT)'
                   ,
                    'Dшт (Ш) (ADAPT)']]
    new_result.columns = ['К. напор ' + i , 'К. мощность ' + i,
                          'Рлин ТМ (Ш) (ADAPT) ' + i,
                          'Линейное давление (СУ) (ADAPT) ' + i,
                          'Dшт (Ш) (ADAPT) ' + i]
    wells_result = wells_result.join(new_result, how = 'outer')

In [ ]:
calibr_head = wells_result[wells_result.columns[wells_result.columns.str.contains('давление')]]

In [ ]:
calibr_head = result_and_metrics.make_dimensionless_df(calibr_head)

In [ ]:
traces = pltl_wf.create_traces_list_for_all_columms(calibr_head, 'lines+markers', True)

In [ ]:
pltl_wf.plot_func(traces, 'calibr_head', 'calibr_head.html')